# 1. EDA
En este apartado se pretende analizar los datasets para poder enfocar mejor la limpieza

## 1.1 Importación y carga de datos
Debemos declarar las librerías que usamos y leer el correspondiente archivo de datos

In [1]:
# Importación de librerías
import pandas as pd

# Lectura dataset
df = pd.read_csv('../EncuestasSatisfaccionSucio.csv')

## 1.2 Configuración de Pandas
Para poder leer bien los resultados de las ejecuciones, vamos a configurar tanto el número máximo de columnas como el número máximo de filas

In [2]:
# Número máximo de filas a mostrar
pd.set_option('display.max_rows', None)

# Número máximo de columnas a mostrar
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 2000)

## 1.3 Descripción general del dataset
Para poder conocer ciertas características relevantes del dataset, como el número de instancias (filas) y características (columnas) procederemos a usar diferentes funciones de Pandas


In [3]:
# Descripción de parámetros generales -> count, mean, std, min, 25%, 50%, 75%, max
print(df.describe())

# Número de filas y columnas del dataset
print("\n")
print("Número de filas: ", df.shape[0])
print("Número de columnas: ", df.shape[1])
print("\n")

# Para saber el tipo de variable de cada columna
print(df.info())

                ID  PUNTUACION_ACCESIBILIDAD  PUNTUACION_CALIDAD  AreaRecreativaID
count  7000.000000               7000.000000         7000.000000      7.000000e+03
mean   3500.500000                  2.976857            2.995286      1.545206e+06
std    2020.870275                  1.416548            1.399429      2.802642e+06
min       1.000000                  1.000000            1.000000      1.665700e+04
25%    1750.750000                  2.000000            2.000000      1.742200e+04
50%    3500.500000                  3.000000            3.000000      1.811450e+04
75%    5250.250000                  4.000000            4.000000      4.515310e+05
max    7000.000000                  5.000000            5.000000      9.503906e+06


Número de filas:  7000
Número de columnas:  6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    ------------

## 1.4 Observación inicial del dataset
Vamos a mostrar 30 entradas para poder observar cómo es realmente por dentro el dataset

In [4]:
# Mostramos las 10 primeras filas, 10 filas aleatorias y las 10 últimas
print("   ----------    10 primeras filas    ----------    ")
print("\n")
print(df.head(10))
print("\n")
print("   ----------    10 filas aleatorias    ----------    ")
print("\n")
print(df.sample(10))
print("\n")
print("   ----------    10 últimas filas    ----------    ")
print("\n")
print(df.tail(10))

   ----------    10 primeras filas    ----------    


   ID  PUNTUACION_ACCESIBILIDAD  PUNTUACION_CALIDAD COMENTARIOS  AreaRecreativaID       FECHA
0   1                         2                   1     Regular             18510  27-03-2021
1   2                         5                   5   Excelente             52143  25/03/2023
2   3                         1                   1  Deficiente           6907760  19-03-2022
3   4                         2                   1     Regular           9187084  2021-06-25
4   5                         3                   2   Aceptable             17256  28-12-2023
5   6                         5                   5   Excelente             18570  2021-12-09
6   7                         1                   1  Deficiente             17647  2021/02/22
7   8                         1                   2     Regular             52103  12-15-2022
8   9                         1                   1  Deficiente             52195  2021/10/07
9  10

## 1.5 Revisión de valores nulos
Como ya se ha visto en el anterior apartado (info), podemos observar los valores nulos de esa forma. Pero se puede observar de una forma más visual en esta sección y, además, hay que tener en cuenta valores como el cero que también pueden considerarse nulos.

In [5]:
# Para poder saber el número de valores faltantes
print("    ----------    Valores faltantes    ----------    ")
print("\n")
print(df.isnull().sum())
print("\n")

# Para poder saber el número de ceros en cada columna
print("    ----------    Valores cero    ----------    ")
print("\n")
print((df == 0).sum())

    ----------    Valores faltantes    ----------    


ID                          0
PUNTUACION_ACCESIBILIDAD    0
PUNTUACION_CALIDAD          0
COMENTARIOS                 0
AreaRecreativaID            0
FECHA                       0
dtype: int64


    ----------    Valores cero    ----------    


ID                          0
PUNTUACION_ACCESIBILIDAD    0
PUNTUACION_CALIDAD          0
COMENTARIOS                 0
AreaRecreativaID            0
FECHA                       0
dtype: int64


No encontramos valores nulos, por lo que no se hará limpieza de esto

## 1.6 Identificación de fechas no estandarizadas
Se deben identificar las fechas que no se encuentran en el formato adecuado para MongoDB (DD/MM/YYYY)

In [6]:
# Vamos a mostrar algunas fechas para poder observar en qué formato están
print("   ----------    Fechas    ----------    ")
print("\n")
print(df['FECHA'].sample(20))

   ----------    Fechas    ----------    


6679    2021-03-09
3995    24-10-2023
4160    09/04/2023
5249    2023-10-03
2543    2021/08/03
1924    2022/04/28
5678    11-24-2023
1577    12-18-2021
3433    16/12/2023
4428    2021/07/26
2712    2021-03-29
2105    08-20-2022
5690    26-05-2023
2753    2021/09/02
321     01-19-2021
959     25/03/2021
6362    2021-04-04
99      01-14-2021
4721    17/06/2023
2547    07-10-2021
Name: FECHA, dtype: object


Como se puede apreciar, las fechas se encuentran en diversos formatos que deben ser homogeneizados

## 1.7 Identificación de registros duplicados
Debemos validar que no existen filas iguales que ensucien el dataset, sobre todo estando pendiente de duplicaciones de la clave primaria

In [7]:
# Ver las filas duplicadas
print("   ----------    Filas duplicadas    ----------    ")
print("\n")
print(df[df.duplicated()])
print("\n")

# Número de filas duplicadas
print("Número de filas duplicadas: ", df.duplicated().sum())
print("\n")

# Filtrar filas que tienen el mismo ID
duplicados = df.groupby('ID').filter(lambda x: len(x) > 1)

# Ordenar por NIF para que las filas con el mismo NIF se visualicen una encima de la otra
duplicados = duplicados.sort_values(by='ID')

# Mostrar las filas con la misma PK
print("   ----------    Filas con el mismo ID    ----------    ")
print("\n")
print(duplicados)
print("\n")
print("Número de filas con el mismo ID: ", duplicados.shape[0])

   ----------    Filas duplicadas    ----------    


Empty DataFrame
Columns: [ID, PUNTUACION_ACCESIBILIDAD, PUNTUACION_CALIDAD, COMENTARIOS, AreaRecreativaID, FECHA]
Index: []


Número de filas duplicadas:  0


   ----------    Filas con el mismo ID    ----------    


Empty DataFrame
Columns: [ID, PUNTUACION_ACCESIBILIDAD, PUNTUACION_CALIDAD, COMENTARIOS, AreaRecreativaID, FECHA]
Index: []


Número de filas con el mismo ID:  0


No existen filas duplicadas, por lo que no es necesario limpiar en este paso

## 1.8 Búsqueda de errores tipográficos
Hay ciertos atributos de texto que pueden contar con determinados errores tipográficos que deben ser solucionados, como los nombres de áreas, juegos, usuarios y ubicaciones

En este dataset no encontramos ningún atributo similar, por lo que este paso no se realizará

## 1.9 Identificación de valores enum fuera de campo
Hay ciertos atributos que solo deben poseer ciertos valores (como Operativo-NoOperativo). Hace falta identificar aquellos valores de ese campo fuera de la norma

No hay atributos Enum, por lo que este paso se saltará

## 1.10 Validación de las coordenadas y otros campos geoespaciales
Hay algunas veces en las que los códigos postales no respetan la identificación de Madrid (280..) o el formato, ya sean códigos postales u otros atributos de geolocalización

En este dataset no encontramos valores de localización, por lo que este paso no se realizará

## 1.11 Identificación de unidades de medida en un formato no estandarizado
Se deben identificar las filas que no posean un formato estándar

En este dataset no encontramos valores de unidades de medida, por lo que este paso no se realizará

## 1.12 Otros atributos a corregir
En esta sección se mencionarán aquellos atributos que también deban ser limpiados por errores

### 1.12.1 Validación de IDs
Para este paso debemos comprobar que todos los IDs son enteros, ya que en el paso de los duplicados ya hemos comprobado esto

In [8]:
# Debemos validar que todos los IDs son únicos y enteros
print("   ----------    Validación de IDs    ----------    ")
print("\n")
print('El ID es un entero: ', df['ID'].apply(lambda x: x.is_integer()).all())

   ----------    Validación de IDs    ----------    


El ID es un entero:  True


Todos los IDs son enteros, por lo que no se precisará su limpieza

### 1.12.2 Consideraciones extras de Fechas
Cabe recalcar que si el campo de fechas no posee un formato esperado, las funciones de limpieza darán error, por lo que no hace falta hacer ahora las comprobaciones de sus valores para determinar si son correctos.

### 1.12.3 Validación de campos con formato Enum
Aunque no sean Enum como tal, hay columnas que se pueden validar siguiendo un proceso parecido al que usamos al tener Enums. En concreto, la columna de 'PUNTUACION_ACCESIBILIDAD', 'PUNTUACION_CALIDAD' Y 'COMENTARIOS' se puede validar viendo sus posibles valores y comprobando si tienen sentido

In [9]:
# Muestra todos los valores distintos de la columna 'PUNTUACION_ACCESIBILIDAD'
print("   ----------    Valores distintos de la columna 'PUNTUACION_ACCESIBILIDAD'    ----------    ")
print("\n")
print(df['PUNTUACION_ACCESIBILIDAD'].unique())
print("\n")

# Muestra todos los valores distintos de la columna 'PUNTUACION_CALIDAD'
print("   ----------    Valores distintos de la columna 'PUNTUACION_CALIDAD'    ----------    ")
print("\n")
print(df['PUNTUACION_CALIDAD'].unique())
print("\n")

# Muestra todos los valores distintos de la columna 'COMENTARIOS'
print("   ----------    Valores distintos de la columna 'COMENTARIOS'    ----------    ")
print("\n")
print(df['COMENTARIOS'].unique())

   ----------    Valores distintos de la columna 'PUNTUACION_ACCESIBILIDAD'    ----------    


[2 5 1 3 4]


   ----------    Valores distintos de la columna 'PUNTUACION_CALIDAD'    ----------    


[1 5 2 4 3]


   ----------    Valores distintos de la columna 'COMENTARIOS'    ----------    


['Regular' 'Excelente' 'Deficiente' 'Aceptable' 'Muy bueno']


Como poseen valores que tienen sentido, no hace falta tener que limpiarlo al no tener nulos

### 1.12.4 Consideraciones extras de AreaRecreativaID
Se ha observado que los IDs presentes poseen un formato idéntico a los IDs del dataset de áreas (como debe ser), así que debemos validar que todos los valores de esta columna se encuentran allí para confirmar que se pueden enlazar ambos datasets

In [10]:
# Comprobar que todos los valores de la columna 'AreaRecreativaID' existen en la columna 'ID' del dataset 'AreasSucio.csv'
areas = pd.read_csv('../AreasSucio.csv')
print("   ----------    Comprobación de valores en 'AreaRecreativaID'    ----------    ")
print("\n")
print("Todos los valores se encuentran en el otro dataset: ", df['AreaRecreativaID'].isin(areas['ID']).all())

   ----------    Comprobación de valores en 'AreaRecreativaID'    ----------    


Todos los valores se encuentran en el otro dataset:  True


Como todos los valores se encuentran en el otro dataset, no hace falta hacer transformaciones

# 2. Limpieza de los datasets
En este apartado se realizará la limpieza según la información obtenida en el análisis exploratorio de datos:
- Se debe corregir las fechas y dejarlas en un formato homogéneo

## 2.1 Limpieza de FECHA
Como en otros datasets, debemos dejar todas las fechas en formato apto para MongoDB (DD/MM/YYYY)

In [11]:
# Leemos toda la columna de fechas y almacenamos todas las fechas en una lista
fechas = df['FECHA'].tolist()

# Ahora, pasamos por todas las fechas y corregimos las fechas que están mal escritas
fechas_corregidas = []
año_al_final = False

for fecha in fechas:
    # Si fecha contiene un guion, hacer fecha.split('-') y si fecha contiene una barra, hacer fecha.split('/')
    if '-' in fecha:
        fecha_split = fecha.split('-')
    elif '/' in fecha:
        fecha_split = fecha.split('/')
    # Condiciones para saber el día, mes y año
    # Encontrar el año
    if (int(fecha_split[0])>31):
        año = fecha_split[0]
        año_al_final = False
    elif (int(fecha_split[2])>31):
        año = fecha_split[2]
        año_al_final = True
    else:
        print("No se ha encontrado el año")
        fechas_corregidas.append("fecha incorrecta")
        break
    # Encontrar el día y el mes
    # Si el año está al final, no comprobamos el primer caracter
    if año_al_final:
        if (int(fecha_split[0])<32 and int(fecha_split[0])>12):
            dia = fecha_split[0]
            mes = fecha_split[1]
        elif (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[0]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el primero
        else:
            dia = fecha_split[0]
            mes = fecha_split[1]
    else:
        if (int(fecha_split[1])<32 and int(fecha_split[1])>12):
            dia = fecha_split[1]
            mes = fecha_split[2]
        elif (int(fecha_split[2])<32 and int(fecha_split[2])>12):
            dia = fecha_split[2]
            mes = fecha_split[1]
        # Si no hay ningun número entre el 13 y el 31, se asume que el mes es el segundo siempre y el día es el tercero
        else:
            dia = fecha_split[2]
            mes = fecha_split[1]
    fechas_corregidas.append(dia+"/"+mes+"/"+año)
    
# Ahora, cambiamos toda la columna de FECHAS_INTERVENCION por las fechas corregidas
df['FECHA'] = fechas_corregidas

# Mostramos las fechas corregidas
print("   ----------    Columna de fechas corregidas    ----------    ")
print("\n")
print(df['FECHA'].sample(20))

   ----------    Columna de fechas corregidas    ----------    


1893    29/03/2021
243     07/07/2023
961     23/11/2022
4067    17/06/2022
5617    15/06/2021
615     19/01/2022
332     05/09/2023
6116    28/10/2023
3224    09/02/2021
2619    14/11/2022
621     21/03/2022
3741    25/12/2022
2385    16/11/2023
1928    16/10/2022
3817    27/02/2022
2919    19/08/2022
4670    06/06/2023
4020    24/01/2023
1227    26/02/2021
5117    13/03/2022
Name: FECHA, dtype: object


## 2.2 Mostrar Dataset Limpio y guardar CSV
Vamos a mostrar algunas filas del dataset limpio para validar que todo está OK y guardamos el Dataset

In [12]:
# Enseñamos 20 filas aleatorias para ver cómo quedan
print("   ----------    10 filas aleatorias    ----------    ")
print("\n")
print(df.sample(10))
print("\n")

# Guardamos el dataset limpio
df.to_csv('../EncuestasSatisfaccionLimpio.csv', index=False)
print("\n")
print("Dataset limpio guardado")

   ----------    10 filas aleatorias    ----------    


        ID  PUNTUACION_ACCESIBILIDAD  PUNTUACION_CALIDAD COMENTARIOS  AreaRecreativaID       FECHA
6434  6435                         3                   4   Muy bueno             17795  26/06/2021
2520  2521                         3                   2   Aceptable             18047  03/11/2021
1390  1391                         1                   1  Deficiente             17488  27/01/2023
2172  2173                         4                   5   Excelente             17301  07/09/2022
5068  5069                         4                   5   Excelente             52089  26/10/2022
4466  4467                         3                   2   Aceptable             17470  02/04/2021
4880  4881                         5                   4   Excelente           8799606  02/01/2023
1579  1580                         4                   5   Excelente             52200  14/02/2022
2197  2198                         4                